# 9. 변수 범위, Closure, Decorator

## (1) 전역변수 vs 지역변수
- **참고)** https://wikidocs.net/62

- 예제 1

In [1]:
def func_v1(a): 
    print(a)
    print(b)
    
func_v1(5)  # Error: b가 정의되어있지 않은데 b를 출력하라고 명령헀기 때문

5


NameError: name 'b' is not defined

- 예제 2

In [2]:
b = 10  # ---> *** 전역변수 ***

def func_v2(a): 
    print(a)
    print(b)

func_v2(5)

5
10


- 예제 3 <br> 
 **UnboundLocalError:** 할당되기 전에 지역변수 'b'가 참조되었다.<br> 
함수 내에 b가 있는 것은 체크가 되었는데, b에 값을 할당하기 전에 b를 출력했기 때문에 에러가 난다. 같은 이름의 전역변수와 지역변수가 있으면 **함수 내에서는 지역변수가 우선**이며, 런타임의 인터프리터가 함수 내에 b가 있는 것을 체크했지만, 할당은 위에서부터 아래로 이루어지기 때문이다. 

In [3]:
b = 10 

def func_v3(a): 
    print(a)
    print(b)
    b = 5 
    
func_v3(5)

5


UnboundLocalError: local variable 'b' referenced before assignment

In [4]:
# 증명 
from dis import dis 
print(dis(func_v3))

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (5)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
None


## (2) 클로저 Closure
1. 반환되는 내부 함수에 대해서 선언된 연결 정보를 가지고 참조하는 방식 
2. 반환 당시 함수의 유효범위를 벗어난 변수 또는 메소드에 직접 접근이 가능 

- __개별적인 계산__

In [7]:
a = 10 

print(a + 10)
print(a + 100)
print(sum(range(1, 51)))
print(sum(range(51, 101)))

20
110
1275
3775


- __결과 누적__
    - 1. 클래스 이용

In [8]:
class Averager(): 
    def __init__(self): 
        self._series = []
        
    def __call__(self, v): 
        self._series.append(v)
        print('class >>> {} / {}'.format(self._series, len(self._series)))
        return sum(self._series) / len(self._series)

In [10]:
avg_cls = Averager()

print(avg_cls(15))
print(avg_cls(35))
print(avg_cls(40))

class >>> [15] / 1
15.0
class >>> [15, 35] / 2
25.0
class >>> [15, 35, 40] / 3
30.0


- __결과 누적__
    - 클로저 이용
        1. 전역변수 사용 감소
        2. 디자인 패턴 적용

In [11]:
def closure_avg1(): 
    # free variable
    # 클로저 영역 
    series = []
    def averager(v):
        # 여기에 series가 없지만 계속 참조하고 있음 
        # series = [] 여기에 쓴다면 계속 초기화된 후 append됨 
        series.append(v)
        print('def1 >>> {} / {}'.format(series, len(series)))
        return sum(series) / len(series)
    return averager # 함수 자체를 리턴 averager() 이게 아님 

In [13]:
avg_closure1 = closure_avg1()

print(avg_closure1(15))
print(avg_closure1(35))
print(avg_closure1(40))

def1 >>> [15] / 1
15.0
def1 >>> [15, 35] / 2
25.0
def1 >>> [15, 35, 40] / 3
30.0


In [22]:
print(dir(avg_closure1)) # __code__
print()
print(dir(avg_closure1.__code__))  # co_freevars
print()
print(avg_closure1.__code__.co_freevars)  # series 
print()
print(dir(avg_closure1.__closure__[0]))
print()
print(dir(avg_closure1.__closure__[0].cell_contents[1]))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'co_argcount', 'co_cellvars', 'co_code', 'co_consts', 'co_filename', 'co_firstlineno', 'co_flags', 'co_freevars', 'co_kwonlyargcount', 'co_lnotab', 'co_name', 'co_names', 'co_nlocals', 'co_stacksize', 'co_varnames']

('

- 잘못된 클로저 사용 예

In [23]:
def closure_avg2(): 
    # free variable 
    # 클로저 영역 
    cnt = 0 
    total = 0 
    
    def averager(v): 
        cnt += 1 
        total += v
        print("def2 >>> {}  {}".format(total, cnt))
        return total / cnt 
    return averager 

In [24]:
avg_closure2 = closure_avg2()

print(avg_closure2(15))  # Error 

UnboundLocalError: local variable 'cnt' referenced before assignment

- 해결방법

In [25]:
def closure_avg2(): 
    # free variable
    # 클로저 영역 
    cnt = 0 
    total = 0 
    def averager(v): 
        nonlocal cnt, total 
        cnt += 1 
        total += v 
        print("def2 >>> {} / {}".format(total, cnt))
        return total / cnt
    return averager

In [27]:
avg_closure2 = closure_avg2()

print(avg_closure2(15))
print(avg_closure2(35))
print(avg_closure2(40))

def2 >>> 15 / 1
15.0
def2 >>> 50 / 2
25.0
def2 >>> 90 / 3
30.0


## (3) 데코레이터 Decorator 
- **장점**
    1. 중복제거, 코드간결 
    2. 클로저보다 문법 간결 
    3. 조합해서 사용할 때 용이 
    
    
- **단점**
    1. 디버깅이 어려움 
    2. 에러의 모호함

In [29]:
import time 

def perf_clock(func): 
    def perf_clocked(*args): 
        # 시작 시간 
        st = time.perf_counter() 
        result = func(*args)
        # 종료 시간 
        et = time.perf_counter() - st 
        # 함수명 
        name = func.__name__ 
        # 매개변수 
        arg_str = ','.join(repr(arg) for arg in args)
        # 출력 
        print('Result: {%0.5fs} %s(%s) -> %r' % (et, name, arg_str, result))
        return result 
    return perf_clocked 

- 데코레이터 미사용 

In [31]:
def time_func(seconds): 
    time.sleep(seconds)

def sum_func(*numbers): 
    return sum(numbers)

def fact_func(n):
    return 1 if n < 2 else n * fact_func(n-1)

In [35]:
non_deco1 = perf_clock(time_func)
non_deco2 = perf_clock(sum_func)
non_deco3 = perf_clock(fact_func)

In [36]:
print(non_deco1, non_deco1.__code__.co_freevars, sep=' || ')
print(non_deco1, non_deco2.__code__.co_freevars, sep=' || ')
print(non_deco1, non_deco3.__code__.co_freevars, sep=' || ')

<function perf_clock.<locals>.perf_clocked at 0x000002679FDCC730> || ('func',)
<function perf_clock.<locals>.perf_clocked at 0x000002679FDCC730> || ('func',)
<function perf_clock.<locals>.perf_clocked at 0x000002679FDCC730> || ('func',)


In [37]:
print("*" * 40, 'Called Non Deco -> time_func')
print(non_deco1(2))
print("*" * 40, 'Called Non Deco -> sum_func')
print(non_deco2(100, 200, 300, 400))
print("*" * 40, 'Called Non Deco -> fact_func')
print(non_deco3(100))

**************************************** Called Non Deco -> time_func
Result: {2.00739s} time_func(2) -> None
None
**************************************** Called Non Deco -> sum_func
Result: {0.00000s} sum_func(100,200,300,400) -> 1000
1000
**************************************** Called Non Deco -> fact_func
Result: {0.00009s} fact_func(100) -> 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000


- 데코레이터 사용

In [38]:
@perf_clock
def time_func(seconds): 
    time.sleep(seconds)
    
@perf_clock
def sum_func(*numbers):
    return sum(numbers)

@perf_clock
def fact_func(n): 
    return 1 if n < 2 else n * fact_func(n-1)

In [40]:
print("*" * 40, 'Called Non Deco -> time_func')
print(time_func(2))
print("*" * 40, 'Called Non Deco -> sum_func')
print(sum_func(100, 200, 300, 400))
print("*" * 40, 'Called Non Deco -> fact_func')
print(fact_func(5))

**************************************** Called Non Deco -> time_func
Result: {1.99998s} time_func(2) -> None
None
**************************************** Called Non Deco -> sum_func
Result: {0.00000s} sum_func(100,200,300,400) -> 1000
1000
**************************************** Called Non Deco -> fact_func
Result: {0.00000s} fact_func(1) -> 1
Result: {0.00005s} fact_func(2) -> 2
Result: {0.00006s} fact_func(3) -> 6
Result: {0.00008s} fact_func(4) -> 24
Result: {0.00009s} fact_func(5) -> 120
120
